In [ ]:
import json
import os
import torch
import re
import asyncio
import nest_asyncio
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from pyngrok import ngrok

# ✅ Google Colab Check
IN_COLAB = "google.colab" in str(get_ipython())

# ✅ Initialize FastAPI (Define 'app' First)
app = FastAPI()

# ✅ Allow requests from your frontend
origins = [
    "https://generative-info-system.web.app",
    "http://localhost:3000",  # (Optional: For local testing)
]

# ✅ Define Paths
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    dataset_folder = "/content/drive/My Drive/DATASETS"
    chroma_db_path = "/content/drive/My Drive/chroma_db"
else:
    dataset_folder = "./DATASETS"
    chroma_db_path = "./chroma_db"

# ✅ Load Multiple Datasets
dataset_files = [
    "gis_dataset.json",
    "conversational_dataset.json",
    "library_dataset.json",
    "office_locations.json"
]

datasets = []
for file in dataset_files:
    dataset_path = os.path.join(dataset_folder, file)
    try:
        with open(dataset_path, "r", encoding="utf-8") as f:
            dataset = json.load(f)
            datasets.extend(dataset)  # ✅ Append all datasets into one list
        print(f"✅ Loaded {file}, Entries: {len(dataset)}")
    except FileNotFoundError:
        print(f"❌ Error: Dataset file '{file}' not found.")

print(f"✅ Total Entries Loaded: {len(datasets)}")

# Allow requests from your frontend
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# ✅ Fix for Colab's event loop
nest_asyncio.apply()

# ✅ Setup ngrok (Replace Token)
NGROK_AUTH_TOKEN = "2tNiI33TgOCzv7votp0JxpAeE4u_6snykGnd8yDG3QkzqwvZS"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# ✅ Load Language Model
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Ensure correct device selection
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto").to(device)

# ✅ Load ChromaDB (Vector Store)
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_db = Chroma(embedding_function=embedding_model, persist_directory=chroma_db_path)

# ✅ Inject Multiple Datasets into ChromaDB (Improved)
documents = []
metadata = []
for entry in datasets:
    if "input" in entry and "output" in entry:  # Ensure valid format
        documents.append(f"Q: {entry['input']} A: {entry['output']}")
        metadata.append({"source": "dataset"})

vector_db.add_texts(documents, metadatas=metadata)
print("✅ All datasets added to ChromaDB!")

# ✅ Initialize Chatbot Conversation
def initialize_conversation():
    """Set system message for Santi, the UA chatbot."""
    system_message = """
    You are Santi, the AI assistant for the University of Antique.
    Your role is to provide accurate information about UA, including courses, history, enrollment, and services.
    Always give direct, relevant answers based on UA's official information.
    """
    return [{"role": "system", "content": system_message}]

# ✅ Define Input Schema
class QueryInput(BaseModel):
    query: str

@app.post("/ask")
async def ask_model(data: QueryInput):
    query = data.query
    results = vector_db.similarity_search(query, k=5)  # Increased top-k to 3 for better accuracy

    # ✅ Handle empty search results
    if not results:
        return {"response": "I couldn't find a relevant answer in the database. Please contact the university for official information."}

    context = "\n".join([doc.page_content for doc in results])

    # ✅ Confidence Threshold
    if len(context.split()) < 20:  # If retrieved text is too short, discard it
        return {"response": "I'm sorry, but I couldn't find enough information on that topic."}

    prompt = f"""You are Santi, the University of Antique AI Chatbot.
    Use the provided context to answer the question accurately and concisely.

    Context:
    {context}

    Question: {query}

    Answer:"""

    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)

    token_length = len(query.split())  # Count words in query
    max_tokens = min(50 + (token_length * 2), 250)  # Dynamically adjust response length

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,  # Adjusted dynamically
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # ✅ Extract answer properly
    answer_match = re.search(r"Answer:\s*(.*?)(?:\n|$)", response, re.DOTALL)
    final_response = answer_match.group(1).strip() if answer_match else response

    return {"response": final_response}


# ✅ Start ngrok tunnel
public_url = ngrok.connect(8000).public_url
print(f"🚀 Public API URL: {public_url}")

# ✅ Run FastAPI in Colab
import uvicorn

if __name__ == "__main__":
    asyncio.run(uvicorn.run(app, host="0.0.0.0", port=8000))


Mounted at /content/drive
✅ Loaded gis_dataset.json, Entries: 63
✅ Loaded conversational_dataset.json, Entries: 105
✅ Loaded library_dataset.json, Entries: 33
✅ Loaded office_locations.json, Entries: 46
✅ Total Entries Loaded: 247


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
<ipython-input-1-5f4ea7526c31>:83: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-t

✅ All datasets added to ChromaDB!


INFO:     Started server process [3315]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🚀 Public API URL: https://8d96-34-75-174-68.ngrok-free.app
